In [1]:
duel_path="/home/ifajcik/code/lm-evaluation-harness/benczechmark_leaderboard/duel_matrices"
import csv
import os
import pandas as pd
import json
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, FactorRange
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.layouts import column
import seaborn as sns

# Ensure output is displayed in the notebook
output_notebook()

PREFIX="/home/ifajcik/code/lm-evaluation-harness/benczechmark_leaderboard"
results_overall=pd.read_csv("leaderboard_data/Leaderboard - Overall.csv")
results_detailed=pd.read_csv("leaderboard_data/Leaderboard - Overall with details.csv")
TOURNAMENT_PATH = "leaderboard_data/tournament.json"
NAME_PATH="leaderboard_data/submission_id_to_model_title.json"
METADATA_PATH = PREFIX+"/leaderboard/metadata.json"
with open(METADATA_PATH) as f:
    m = json.load(f)
with open(TOURNAMENT_PATH) as f:
    tournament = json.load(f)
with open(NAME_PATH) as f:
    name_map = json.load(f)



duel_matrix = dict()
tasks = list(tournament['speakle_Bielik-_a5a94ef_d956c5d'][list(tournament[list(tournament.keys())[0]].keys())[0]].keys())
for task in tasks:
    task = task.replace('benczechmark_', '')
    if task not in duel_matrix:
        duel_matrix[task]=dict()
    for model_id in tournament.keys():
        translated_model_id = name_map[model_id]
        if translated_model_id.startswith("Qwen/Qwen2.5"):
            continue
        if model_id not in duel_matrix:
            duel_matrix[task][translated_model_id] = dict()
        duel_matrix[task][translated_model_id][translated_model_id]=False
        for model_against in tournament[model_id].keys():
            if model_id == model_against:
                continue
            
            translated_model_against_id = name_map[model_against]
            duel_matrix[task][translated_model_id][translated_model_against_id]=tournament[model_id][model_against]['benczechmark_'+task]['significant']

all_matrices=duel_matrix

# # Paths to files
# OUTFNAME = PREFIX+"/hf_comparison.csv"
# METADATA_PATH = PREFIX+"/leaderboard/metadata.json"

# # Load metadata to get task categories
# with open(METADATA_PATH, "r") as f:
#     metadata = json.load(f)
    
# # Load and average duel matrices
# csv_files = [f for f in os.listdir(duel_path) if f.endswith('.csv')]
# all_matrices = dict()

# for file in csv_files:
#     matrix = pd.read_csv(os.path.join(duel_path, file), index_col=0)
#     matrix = matrix>0
#     matrix=matrix.rename(
#        index=lambda x: x.replace('results_hf_', '').replace('results_', '').replace('.json', ''),
#        columns=lambda x: x.replace('results_hf_', '').replace('results_', '').replace('.json', '')
#     )
#     all_matrices[file.replace("duel_matrix_benczechmark_","").replace(".csv", "")]=matrix

# # Read the CSV into a DataFrame
# df = pd.read_csv(OUTFNAME)

# # Ignore the first column (WinScore)
# df = df.drop(columns=["MicroWinScore"])
# # Rename model names in the first column
# df[df.columns[0]] = df[df.columns[0]].str.replace('results_hf_', '').str.replace('results_', '').str.replace('.json', '')

Loading BokehJS ...

In [2]:

ldb_records={}
with open("leaderboard_data/Leaderboard - Overall.csv", mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['Model'].startswith("Qwen/Qwen2.5"):
            continue
        ldb_records[row['Model']] = row
        
sizes = {model:float(ldb_records[model]['# θ (B)']) for model in ldb_records.keys()}
sizes


{'meta-llama/Meta-Llama-3.1-405B-Instruct': 406.0,
 'meta-llama/Meta-Llama-3.1-70B-Instruct': 70.6,
 'meta-llama/Meta-Llama-3.1-70B': 70.6,
 'Qwen/Qwen2-72B-Instruct': 72.7,
 'mistralai/Mixtral-8x22B-Instruct-v0.1': 141.0,
 'AMead10/c4ai-command-r-08-2024-awq': 32.3,
 'CohereForAI/aya-23-35B': 35.0,
 'meta-llama/Meta-Llama-3.1-8B-Instruct': 8.03,
 'mistralai/Mistral-Nemo-Instruct-2407': 12.2,
 'mistralai/Mixtral-8x7B-Instruct-v0.1': 46.7,
 'speakleash/Bielik-11B-v2.3-Instruct': 11.2,
 'NousResearch/Hermes-3-Llama-3.1-8B': 8.03,
 'meta-llama/Meta-Llama-3.1-8B': 8.03,
 'meta-llama/Meta-Llama-3-8B-Instruct': 8.03,
 'Qwen/Qwen2-7B': 7.62,
 'google/gemma-2-9b': 9.24,
 'google/gemma-2-9b-it': 9.24,
 'Qwen/Qwen2-7B-Instruct': 7.62,
 'ibm-granite/granite-3.0-8b-base': 8.17,
 'CohereForAI/aya-23-8B': 8.03,
 'mistralai/Mistral-7B-Instruct-v0.3': 7.25,
 'meta-llama/Llama-3.2-3B': 3.21,
 'meta-llama/Llama-3.2-3B-Instruct': 3.21,
 'ibm-granite/granite-3.0-8b-instruct': 8.17,
 'google/gemma-2-2b-it'

In [3]:

task_to_category = {}
for task, details in m['tasks'].items():
    task_to_category[task.replace('benczechmark_', '')] = details['category']
task_to_category
    

{'propaganda_argumentace': 'NLI',
 'propaganda_fabulace': 'NLI',
 'propaganda_nazor': 'NLI',
 'propaganda_strach': 'NLI',
 'propaganda_zamereni': 'NLI',
 'propaganda_demonizace': 'NLI',
 'propaganda_lokace': 'NLI',
 'propaganda_relativizace': 'NLI',
 'propaganda_vina': 'NLI',
 'propaganda_zanr': 'NLI',
 'propaganda_emoce': 'NLI',
 'propaganda_nalepkovani': 'NLI',
 'propaganda_rusko': 'NLI',
 'sentiment_mall': 'Sentiment',
 'sentiment_fb': 'Sentiment',
 'sentiment_csfd': 'Sentiment',
 'grammarerrorcorrection': 'Czech Language Understanding',
 'cs_naturalquestions': 'Factual Knowledge',
 'cs_sqad32': 'Reading Comprehension',
 'cs_triviaQA': 'Factual Knowledge',
 'csfever_nli': 'NLI',
 'ctkfacts_nli': 'NLI',
 'cs_ner': 'NER',
 'history_ir': 'Reading Comprehension',
 'hellaswag': 'Language Modeling',
 'histcorpus': 'Language Modeling',
 'klokan_qa': 'Czech Math Reasoning',
 'cs_court_decisions_ner': 'NER',
 'umimeto_biology': 'Factual Knowledge',
 'umimeto_chemistry': 'Factual Knowledge',


In [4]:
# # Calculate the average matrix
# # Assuming all_matrices is a list of DataFrames
concatenated = pd.concat([pd.DataFrame.from_dict(v).T for v in all_matrices.values()])


# Group by index
grouped = concatenated.groupby(level=0)

# Sum values for each group
sum_df = grouped.sum()

# Calculate the adjusted average
average_matrix = sum_df / (len(all_matrices))

# Step 5: Compute variance matrix
# Subtract mean from each matrix, square the result, and sum the squares
squared_diff_sum = grouped.apply(lambda group: ((group - average_matrix.loc[group.name])**2).sum())

# Step 6: Divide by number of matrices to get the variance
variance_matrix = squared_diff_sum / (len(all_matrices) - 1)


In [5]:
average_matrix
#all_matrices['agree']['meta-llama/Meta-Llama-3.1-405B-Instruct']
#pd.DataFrame.from_dict(all_matrices['agree'])

,AMead10/c4ai-command-r-08-2024-awq,BUT-FIT/CSTinyLlama-1.2B,BUT-FIT/csmpt7b,CohereForAI/aya-23-35B,CohereForAI/aya-23-8B,NousResearch/Hermes-3-Llama-3.1-8B,Qwen/Qwen2-72B-Instruct,Qwen/Qwen2-7B-Instruct,Qwen/Qwen2-7B,Qwen/Qwen2.5-72B,...,meta-llama/Meta-Llama-3-8B-Instruct,microsoft/Phi-3-mini-4k-instruct,mistralai/Mistral-7B-Instruct-v0.3,mistralai/Mistral-Nemo-Instruct-2407,mistralai/Mixtral-8x7B-Instruct-v0.1,mistralai/Mixtral-8x22B-Instruct-v0.1,mosaicml/mpt-7b,speakleash/Bielik-11B-v2.3-Instruct,utter-project/EuroLLM-1.7B,utter-project/EuroLLM-1.7B-Instruct
AMead10/c4ai-command-r-08-2024-awq,0.00,0.88,0.86,0.66,0.78,0.72,0.36,0.72,0.82,0.20,...,0.76,0.96,0.86,0.62,0.72,0.28,0.96,0.48,0.98,0.96
BUT-FIT/CSTinyLlama-1.2B,0.08,0.00,0.10,0.10,0.14,0.14,0.12,0.14,0.14,0.10,...,0.16,0.20,0.16,0.12,0.12,0.10,0.22,0.14,0.30,0.30
BUT-FIT/csmpt7b,0.10,0.64,0.00,0.18,0.16,0.16,0.16,0.20,0.24,0.10,...,0.18,0.40,0.22,0.14,0.16,0.14,0.38,0.14,0.44,0.40
CohereForAI/aya-23-35B,0.06,0.82,0.76,0.00,0.66,0.52,0.22,0.58,0.60,0.04,...,0.48,0.88,0.64,0.50,0.44,0.16,0.84,0.42,0.96,0.98
CohereForAI/aya-23-8B,0.04,0.76,0.68,0.16,0.00,0.26,0.12,0.32,0.34,0.02,...,0.30,0.82,0.42,0.22,0.20,0.06,0.80,0.26,0.82,0.84
NousResearch/Hermes-3-Llama-3.1-8B,0.08,0.78,0.74,0.22,0.46,0.00,0.10,0.52,0.56,0.02,...,0.40,0.90,0.60,0.24,0.22,0.06,0.92,0.36,0.82,0.82
Qwen/Qwen2-72B-Instruct,0.42,0.84,0.84,0.64,0.82,0.82,0.00,0.86,0.88,0.08,...,0.80,0.96,0.86,0.74,0.78,0.38,1.00,0.70,0.98,1.00
Qwen/Qwen2-7B,0.10,0.80,0.66,0.24,0.36,0.18,0.04,0.34,0.00,0.02,...,0.26,0.92,0.38,0.12,0.14,0.04,0.86,0.28,0.78,0.80
Qwen/Qwen2-7B-Instruct,0.10,0.80,0.74,0.28,0.50,0.26,0.06,0.00,0.34,0.00,...,0.26,0.92,0.44,0.18,0.18,0.00,0.82,0.16,0.80,0.80
allenai/OLMo-7B-Instruct-hf,0.00,0.46,0.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.18,0.02,0.00,0.02,0.00,0.30,0.00,0.42,0.36


In [6]:
for k in all_matrices.keys():
    assert all(all_matrices[k][m][m]==False for m in all_matrices[k].keys())

In [13]:
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper, HoverTool

from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, leaves_list

# Bokeh Heatmap Visualization
# def create_heatmap(matrix, title=""):
#     plot_width = 800
#     plot_height = 800
#     n_rows, n_cols = matrix.shape

#     p = figure(width=plot_width, height=plot_height, title=f"{title}Average Win Proportion Across Tasks",
#                x_range=list(matrix.columns), y_range=list(matrix.index),
#                toolbar_location="below", tools="pan,wheel_zoom,box_zoom,reset,save", x_axis_label="Loser", y_axis_label="Winner")

#     # Create the color mapper
#     color_mapper = LinearColorMapper(palette='Viridis256', low=0, high=1)

#     # Flatten the matrix for Bokeh plotting
#     source_data = {
#         'x': [],
#         'y': [],
#         'colors': [],
#         'task_names': [],
#         'scores': [],
#     }
#     for i, (task, model_scores) in enumerate(matrix.iterrows()):
#         for j, score in enumerate(model_scores):
#             source_data['x'].append(matrix.columns[j])
#             source_data['y'].append(task)
#             source_data['colors'].append(score)
#             source_data['task_names'].append(matrix.columns[j])
#             source_data['scores'].append(matrix.loc[task, matrix.columns[j]])

#     source = ColumnDataSource(source_data)

#     # Create the heatmap
#     p.rect(x='x', y='y', width=1, height=1, source=source,
#            line_color=None, fill_color={'field': 'colors', 'transform': color_mapper})

#     # Add color bar
#     #color_bar = ColorBar(color_mapper=color_mapper, width=8, location=(0, 0))
#     #p.add_layout(color_bar, 'right')

#     # Add HoverTool
#     hover = HoverTool()
#     hover.tooltips = [("Loser", "@task_names"), ("Winner", "@y"), ("Avg. Win Score", "@scores")]
#     p.add_tools(hover)


#     p.xgrid.grid_line_color = None
#     p.ygrid.grid_line_color = None
#     p.xaxis.major_label_orientation = "vertical"
#     p.xaxis.axis_label_text_font_style = "normal"
#     p.yaxis.axis_label_text_font_style = "normal"
#     return p

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool, ColorBar, LabelSet
from bokeh.transform import linear_cmap
from bokeh.layouts import column
from bokeh.io import show

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool, ColorBar, LabelSet
from bokeh.transform import linear_cmap
from bokeh.layouts import column
from bokeh.io import show

def create_heatmap(matrix, title="", label_font_size="10pt"):
    plot_width = 1200
    plot_height = 1200
    n_rows, n_cols = matrix.shape

    p = figure(width=plot_width, height=plot_height, title=f"{title}Win Proportion Variance Across Tasks",
               x_range=list(matrix.columns), y_range=list(matrix.index),
               toolbar_location="below", tools="pan,wheel_zoom,box_zoom,reset,save", 
               x_axis_label="Loser", y_axis_label="Winner")
    p.title.text_font_size = "13pt"  # Set the font size to 20 points

    # Create the color mapper
    color_mapper = LinearColorMapper(palette='Viridis256', low=matrix.min().min(), high=matrix.max().max())

    # Flatten the matrix for Bokeh plotting
    source_data = {
        'x': [],
        'y': [],
        'colors': [],
        'task_names': [],
        'scores': [],
    }

    # Populate the source data with x, y, color values (heatmap), and the scores
    for i, (task, model_scores) in enumerate(matrix.iterrows()):
        for j, score in enumerate(model_scores):
            source_data['x'].append(matrix.columns[j])
            source_data['y'].append(task)
            source_data['colors'].append(score)
            source_data['task_names'].append(matrix.columns[j])
            source_data['scores'].append(matrix.loc[task, matrix.columns[j]])

    source = ColumnDataSource(source_data)

    # Create the heatmap with colored rectangles
    p.rect(x='x', y='y', width=1, height=1, source=source,
           line_color=None, fill_color={'field': 'colors', 'transform': color_mapper})

    # Add color bar
    #color_bar = ColorBar(color_mapper=color_mapper, width=8, location=(0, 0))
    #p.add_layout(color_bar, 'right')

    # Add HoverTool
    hover = HoverTool()
    hover.tooltips = [("Loser", "@task_names"), ("Winner", "@y"), ("Avg. Win Score", "@scores")]
    p.add_tools(hover)

    # Add numeric labels to the heatmap (display scores as text in each cell)
    label_data = {
        'x': [],
        'y': [],
        'value': [],
        'text_color': [],  # Dynamic text color (white/black based on score)
    }

    # Add labels for each cell, multiplying by 100 and rounding
    for i, (task, model_scores) in enumerate(matrix.iterrows()):
        for j, score in enumerate(model_scores):
            label_data['x'].append(matrix.columns[j])
            label_data['y'].append(task)
            rounded_score = round(score * 100)  # Multiply by 100 and round to whole numbers
            label_data['value'].append(rounded_score)  # Add rounded score as text
            # Choose the text color based on the score (light/dark)
            if score <= 0.6:
                label_data['text_color'].append('white')
            else:
                label_data['text_color'].append('black')

    label_source = ColumnDataSource(label_data)

    # Create LabelSet to display the numeric scores on the heatmap
    labels = LabelSet(x='x', y='y', text='value', source=label_source,
                      text_color='text_color', text_align='center', text_baseline='middle',
                      text_font_size=label_font_size)  # Use the passed font size parameter

    # Add the labels to the plot
    p.add_layout(labels)

    # Customize the plot appearance
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.axis_label_text_font_style = "normal"
    p.yaxis.axis_label_text_font_style = "normal"

    p.xaxis.major_label_text_font_size = "11pt"  # X-axis tick labels
    p.yaxis.major_label_text_font_size = "11pt"  # Y-axis tick labels


    return p

# # Example usage of the function:
# import pandas as pd
# import numpy as np

# # Create an example matrix (random values between 0 and 1)
# matrix = pd.DataFrame(np.random.rand(5, 5), columns=['Task1', 'Task2', 'Task3', 'Task4', 'Task5'],
#                       index=['Model1', 'Model2', 'Model3', 'Model4', 'Model5'])

# # Call the heatmap function, specifying label font size
# heatmap_plot = create_heatmap(matrix, title="Win Proportion Heatmap", label_font_size="8pt")

# # Show the plot
# show(heatmap_plot)


In [8]:
# row_sums = average_matrix.sum(axis=1)
# sorted_index = row_sums.sort_values().index


sorted_index = sorted(average_matrix.index, key=lambda x: sizes[x], reverse=False)


# Reorder the DataFrame according to sorted inex
average_matrix_sorted = average_matrix.loc[sorted_index, sorted_index]

# Generate the heatmap
def rename_function(s):
    return s[s.index("/")+1:]
average_matrix_sorted.rename(index=rename_function,columns=rename_function, inplace=True)

heatmap_plot = create_heatmap(average_matrix_sorted,label_font_size="9pt")
#from bokeh.plotting import figure, output_file, save
#output_file("duel_plot.html")
show(heatmap_plot)

from bokeh.io import export_svgs
heatmap_plot.output_backend = "svg"
export_svgs(heatmap_plot, filename="duelmatrix_avg.svg")

['duelmatrix_avg.svg']

In [14]:

# Reorder the DataFrame according to the clustering result
variance_matrix_sorted = variance_matrix.loc[sorted_index, sorted_index]

# Generate the heatmap
def rename_function(s):
    return s[s.index("/")+1:]
variance_matrix_sorted.rename(index=rename_function,columns=rename_function, inplace=True)

heatmap_plot = create_heatmap(variance_matrix_sorted,label_font_size="9pt")
#from bokeh.plotting import figure, output_file, save
#output_file("duel_plot.html")
show(heatmap_plot)

from bokeh.io import export_svgs
heatmap_plot.output_backend = "svg"
export_svgs(heatmap_plot, filename="duelmatrix_var.svg")

['duelmatrix_var.svg']

In [10]:
# now only focus on 10b models and smaller
smaller = [
    'olmo7b_instruct', 
    'phi3mini_instruct', 
    'mpt7', 
    'gemma2-2b', 
    'cstllama', 
    'gemma2-2b_instruct', 
    'internlm_instruct', 
    'csmpt', 
    'mistral03_instruct', 
    'qwen2_lm', 
    'aya23_instruct', 
    'qwen2_instruct', 
    'gemma2_instruct', 
    'llama3_instruct', 
    'llama31_lm', 
    'hermes_llama31', 
    'gemma2_lm', 
    'mistral_nemo_instruct', 
    'llama31_instruct'
]
bigger = [
    'aya23_35b_instruct', 
    'mixtral8x7_instruct', 
    'qwen2_70b_instruct', 
    'mixtral8x22_instruct', 
    'llama31_70b_lm', 
    'llama31_70b_instruct', 
    'llama31_405b_instruct'
]
# Calculate the average matrix
# Assuming all_matrices is a list of DataFrames
concatenated_small = pd.concat([m.loc[m.index.isin(smaller), smaller] for m in all_matrices.values()])

# Group by index
grouped_small = concatenated_small.groupby(level=0)

# Sum values for each group
sum_df_small = grouped_small.sum()

average_matrix_small = sum_df_small / (len(all_matrices) - 1)

row_sums_small = average_matrix_small.sum(axis=1)
sorted_index_small = row_sums_small.sort_values().index


# Reorder the DataFrame according to the clustering result
average_matrix_sorted_small = average_matrix_small.loc[sorted_index_small, sorted_index_small]

heatmap_plot = create_heatmap(average_matrix_sorted_small)



#from bokeh.plotting import figure, output_file, save
#output_file("duel_plot.html")
show(heatmap_plot)

AttributeError: 'dict' object has no attribute 'loc'

In [ ]:
smaller = [
    'olmo7b_instruct', 
    'phi3mini_instruct', 
    'mpt7', 
    'gemma2-2b', 
    'cstllama', 
    'gemma2-2b_instruct', 
    'internlm_instruct', 
    'csmpt', 
    'mistral03_instruct', 
    'qwen2_lm', 
    'aya23_instruct', 
    'qwen2_instruct', 
    'gemma2_instruct', 
    'llama3_instruct', 
    'llama31_lm', 
    'hermes_llama31', 
    'gemma2_lm', 
    'mistral_nemo_instruct', 
    'llama31_instruct'
]
x = list(all_matrices.values())[0]
x.loc[x.index.isin(smaller), smaller]

In [ ]:
average_matrix_sorted_filtered = average_matrix_sorted

row_variances = average_matrix_sorted.var(axis=1).sort_values(ascending=False)

# Prepare the data
source = ColumnDataSource(data=dict(index=average_matrix_sorted.index, variance=row_variances))


# Create a figure
p = figure(x_range=average_matrix_sorted.index.astype(str).tolist(), title="Variance Across Rows",
           x_axis_label='Row Index', y_axis_label='Variance', height=400, width=800)

# Add a column plot
p.vbar(x='index', top='variance', width=0.9, source=source)

# Rotate the x-axis labels if needed
p.xaxis.major_label_orientation = 1.2

# Show the plot
show(p)

In [11]:
from bokeh.layouts import gridplot


# Step 1: Group tasks by categories
category_to_tasks = {}
for task, category in task_to_category.items():
    if category not in category_to_tasks:
        category_to_tasks[category] = []
    category_to_tasks[category].append(task)

# Step 2: Compute the average matrix for each category using pandas
category_to_average_matrix = {}
for category, tasks in category_to_tasks.items():
    # Collect the DataFrames for all tasks in this category
    list_of_matrices = [pd.DataFrame.from_dict(all_matrices[task]).T for task in tasks]
    
    # Concatenate the DataFrames
    concatenated = pd.concat(list_of_matrices)
    
    # Group by index and columns and calculate the average
    average_matrix = concatenated.groupby(level=0).mean()


    # Reorder the DataFrame according to the general clustering result
    average_matrix_sorted = average_matrix.loc[sorted_index, sorted_index]

    aaverage_matrix_sorted.rename(index=rename_function,columns=rename_function, inplace=True)

    
    # Store the result
    category_to_average_matrix[category] = average_matrix_sorted

# Step 3: Generate the Bokeh heatmaps for each category
plots = []
for category, avg_matrix in category_to_average_matrix.items():
    heatmap = create_heatmap(avg_matrix, title=category,label_font_size="9pt")
    from bokeh.io import export_svgs
    heatmap.output_backend = "svg"
    export_svgs(heatmap, filename=f"duelmatrix_{category}_avg.svg")

    plots.append(heatmap)

# Step 4: Arrange the plots into a grid
n_cols = 2  # Adjust the number of columns
n_rows = (len(plots) + n_cols - 1) // n_cols  # Calculate number of rows needed
grid = gridplot([plots[i * n_cols:(i + 1) * n_cols] for i in range(n_rows)])

# Step 5: Show the grid of plots
show(grid)

In [ ]:
# Extract tasks and their corresponding categories from metadata
task_to_category = {}
for task, details in metadata['tasks'].items():
    task_to_category[task.replace('benczechmark_', '')] = details['category']
task_to_category

